In [1]:
from background_subtraction.background_subtraction import background_sub
from background_subtraction.background_subtraction import background_sub2
from backprojection.backprojection import backproject_single
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import progressbar

**Combining all images with labels**

In [2]:
#train = []
#test = []
#for comb in combined:
#    if (comb[0] == 'train'):
#        train.append( (comb[1] , comb[2]) )
#    if (comb[0] == 'test'):
#        test.append( (comb[1] , comb[2]) )
#        
#print('(train) Found '+str(len(train))+' out of 1875')
#print('(test) Found '+str(len(test))+' out of 472')

In [3]:
def combine_labels_images():
    with open('label_data_copy.json') as json_file:
        data = json.load(json_file)

    orig_str = '/home/emil/Documents/github/BSc-PRO/dataset3/res_still'
    combined = []
    maps = os.listdir(orig_str)
    for m in maps:
        map_str =orig_str+'/'+m
        classes = os.listdir(map_str)
        for k in range(len(classes)):
            path_str = map_str+'/'+classes[k]
            names = os.listdir(path_str)
            count = 0
            for i in range(len(names)):
                for j in range(len(data)):
                    if names[i] == data[j]['External ID'] and (classes[k] != 'background'):
                        combined.append((m ,path_str+'/'+names[i], data[j] ))
    return combined

In [4]:
def IoU(boxPred, boxGT):
    # Create intersection rectangle
    xIUL = max(boxPred[0], boxGT[0])
    yIUL = max(boxPred[1], boxGT[1])
    xIBR = min(boxPred[2], boxGT[2])
    yIBR = min(boxPred[3], boxGT[3])
    
    # Area of intersection rectangle
    iArea = max(0, xIBR - xIUL + 1) * max(0, yIBR - yIUL + 1)
    # Compute area of boxes
    boxPredArea = (boxPred[2] - boxPred[0] + 1) * (boxPred[3] - boxPred[1] + 1)
    boxGTArea = (boxGT[2] - boxGT[0] + 1) * (boxGT[3] - boxGT[1] + 1)
    iou = iArea / float(boxPredArea + boxGTArea - iArea)
    return iou

In [5]:
def findIndicesForIoU(c):
    x = []
    combined = combine_labels_images()
    for i in range(len(combined)):
        label = combined[i][2]['Label']
        if c in label:
            x.append(i)
            
    return x

# IoU Test For Background Subtraction

In [6]:
mask = cv2.imread('background_subtraction/bgd_mask.jpg', cv2.IMREAD_GRAYSCALE)
background_img = cv2.imread('background_subtraction/avg_background.jpg', cv2.IMREAD_COLOR)

def calculate_IoU(indx):
    combined = combine_labels_images()
    #print('Amount of combined objects: '+str(len(combined)))
    current = combined[indx]

    img = cv2.imread(current[1])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    x = []
    map_str = '/home/emil/Documents/github/BSc-PRO/dataset3/res_still/test'
    classes = os.listdir(map_str)
    for c in classes:
        label = current[2]['Label']
        if c in label:
            ground_truth = c
            (x1, y1) = ( label[c][0]['geometry'][0]['x'],label[c][0]['geometry'][0]['y'] )
            (x2, y2) = ( label[c][0]['geometry'][1]['x'],label[c][0]['geometry'][1]['y'] )
            (x3, y3) = ( label[c][0]['geometry'][2]['x'],label[c][0]['geometry'][2]['y'] )
            (x4, y4) = ( label[c][0]['geometry'][3]['x'],label[c][0]['geometry'][3]['y'] )
            x = [x1, x2, x3, x4]
            y = [y1, y2, y3, y4]

    (xUL, yUL) = (np.min(x),np.min(y))
    (xBR, yBR) = (np.max(x),np.max(y))

    box = [xUL, yUL, xBR, yBR]
    roi, bb = background_sub(img, background_img, mask)
    
    (pXUL, pYUL) = (int(bb[0]), int(bb[1]))
    (pXBR, pYBR) = (int(bb[0] + bb[2]), int(bb[1] + bb[3]))
    boxPred = [pXUL, pYUL, pXBR, pYBR]
    
    (midX, midY) = (pXUL + int((pXBR - pXUL)/2) , pYUL + int((pYBR - pYUL)/2) )
    midPointContained = False
    if midX >=xUL and midX <= xBR and midY >= yUL and midY <= yBR:
        midPointContained = True
    #print('UL: '+str(xUL)+','+str(yUL)+'      BR: '+str(xBR)+','+str(yBR))
    #print('T/F:' + str(midPointContained)+'mid:'+str(midX)+','+str(midY))
    iou = IoU(box, boxPred)
    
    #print("IoU: "+str(iou))
    return iou, midPointContained

In [7]:
def localization_test(ignore=['background']):
    map_str = '/home/emil/Documents/github/BSc-PRO/dataset3/res_still/test'
    classes = os.listdir(map_str)
    for i in range(len(classes)):
        skip = False
        for obj in ignore:
            if classes[i] == obj:
                skip = True
        if (skip):
            continue
        print('Current class: '+str(classes[i]))
        allIndices = findIndicesForIoU(classes[i])
        cSum = 0
        contained = 0
        o50Sum = 0
        it = 0
        bar = progressbar.ProgressBar(max_value=len(allIndices))

        for indx in allIndices:
            iou, midPointContained = calculate_IoU(indx)
            cSum += iou
            if iou >= 0.50:
                o50Sum += 1
            if midPointContained:
                contained += 1
            it += 1
            #bar.update(it)
            if it % 25 == 0:
                print(str(it)+'/'+str(len(allIndices)))

        cSum /= len(allIndices)
        o50Sum /= len(allIndices)
        contained /= len(allIndices)
        print('Average IoU for '+str(classes[i])+': '+str(cSum)+',     Percentage above 0.50 in IoU: '+str(o50Sum)+',     Percentage with midpoint in bb: '+str(contained))

In [8]:
localization_test()

Current class: carrots
25/292
50/292
75/292
100/292
125/292
150/292
175/292
200/292
225/292
250/292
275/292
Average IoU for carrots: 0.8465887905071818,     Percentage above 0.50 in IoU: 1.0,     Percentage with midpoint in bb: 1.0
Current class: ketchup
25/330
50/330
75/330
100/330
125/330
150/330
175/330
200/330
225/330
250/330
275/330
300/330
325/330
Average IoU for ketchup: 0.6217181952293174,     Percentage above 0.50 in IoU: 0.8515151515151516,     Percentage with midpoint in bb: 1.0
Current class: arm
25/297
50/297
75/297
100/297
125/297
150/297
175/297
200/297
225/297
250/297
275/297
Average IoU for arm: 0.7158331251782607,     Percentage above 0.50 in IoU: 0.9629629629629629,     Percentage with midpoint in bb: 0.9831649831649831
Current class: bun
25/334
50/334
75/334
100/334
125/334
150/334
175/334
200/334
225/334
250/334
275/334
300/334
325/334
Average IoU for bun: 0.7875357495536999,     Percentage above 0.50 in IoU: 1.0,     Percentage with midpoint in bb: 1.0
Current cla

|Class             | Average IoU | % Above 0.50 IoU | % With midpoint inside bb |
|:----------------:|:-----------:|:----------------:|:-------------------------:|
| Carrots          | 0.8465      |      1.0000      |        1.0000             |
| Ketchup          | 0.6217      |      0.8515      |        1.0000             |
| Arm              | 0.7158      |      0.9629      |        0.9831             |
| Bun              | 0.7875      |      1.0000      |        1.0000             |
| Cat food beef    | 0.4628      |      0.2491      |        1.0000             |
| Potato           | 0.8569      |      1.0000      |        1.0000             |
| Cat food salmon  | 0.4389      |      0.1287      |        0.9966             |
| **All classes**  | 0.6762      |      0.7465      |        0.9972             |

Total antal billeder: 2347, der bliver fratrukket 201 baggrundsbilleder, dermed 2146 billeder.

In [11]:
cSum = 0
o50Sum = 0
contained = 0
for i in range(2146):
    iou, midPointContained  = calculate_IoU(i)
    cSum += iou
    if iou >= 0.50:
        o50Sum += 1
    if midPointContained:
        contained += 1
    #print(i)
cSum /= 2146
o50Sum /= 2146
contained /= 2146

print('Average of IOU: '+str(cSum)+',      Percentage above 0.50 in IoU: '+str(o50Sum)+',     Percentage with midpoint in bb: '+str(contained))

Average of IOU: 0.6762105040363122,      Percentage above 0.50 in IoU: 0.7465051258154707,     Percentage with midpoint in bb: 0.9972041006523765


**Average over alle 2146 billeder**

IoU = 0.67621

# IoU Test For Backprojection

In [12]:
path = '/home/emil/Documents/github/BSc-PRO/Localization/backprojection/template.jpg'
template = cv2.imread(path, cv2.IMREAD_COLOR)
template = cv2.blur(template, (5, 5))
template_hsv = cv2.cvtColor(template, cv2.COLOR_BGR2HSV)
template_hist = cv2.calcHist(images=[template_hsv],
                             channels=[0, 1],
                             mask=None,
                             histSize=[180, 256],
                             ranges=[0, 180, 0, 256])
cv2.normalize(src=template_hist,
              dst=template_hist,
              alpha=0,
              beta=255,
              norm_type=cv2.NORM_MINMAX)
    
path = '/home/emil/Documents/github/BSc-PRO/Localization/backprojection/bgd_mask.jpg'
mask = cv2.imread(path, cv2.IMREAD_COLOR)

def calculate_IoU(indices):
    combined = combine_labels_images()
    #print('Amount of combined objects: '+str(len(combined)))
    iou = []
    midPointContained = []
    for indx in indices:
        current = combined[indx]

        img = cv2.imread(current[1])

        x = []
        map_str = '/home/emil/Documents/github/BSc-PRO/dataset3/res_still/test'
        classes = os.listdir(map_str)
        for c in classes:
            label = current[2]['Label']
            if c in label:
                ground_truth = c
                (x1, y1) = ( label[c][0]['geometry'][0]['x'],label[c][0]['geometry'][0]['y'] )
                (x2, y2) = ( label[c][0]['geometry'][1]['x'],label[c][0]['geometry'][1]['y'] )
                (x3, y3) = ( label[c][0]['geometry'][2]['x'],label[c][0]['geometry'][2]['y'] )
                (x4, y4) = ( label[c][0]['geometry'][3]['x'],label[c][0]['geometry'][3]['y'] )
                x = [x1, x2, x3, x4]
                y = [y1, y2, y3, y4]

        (xUL, yUL) = (np.min(x),np.min(y))
        (xBR, yBR) = (np.max(x),np.max(y))

        box = [xUL, yUL, xBR, yBR]
        
        regions, bb = backproject_single(hist=template_hist,
                            img=img,
                            bgd_mask=mask)

        (pXUL, pYUL) = (int(bb[0]), int(bb[1]))
        (pXBR, pYBR) = (int(bb[0] + bb[2]), int(bb[1] + bb[3]))
        
        
        boxPred = [pXUL, pYUL, pXBR, pYBR]

        iou.append(IoU(box, boxPred))
        
        
        (midX, midY) = (pXUL + int((pXBR - pXUL)/2) , pYUL + int((pYBR - pYUL)/2) )
        if midX >=xUL and midX <= xBR and midY >= yUL and midY <= yBR:
            midPointContained.append(True)
        else:
            midPointContained.append(False)
        
    
    #print("IoU: "+str(iou))
    return iou, midPointContained

In [13]:
map_str = '/home/emil/Documents/github/BSc-PRO/dataset3/res_still/test'
classes = os.listdir(map_str)
for i in range(len(classes)):
    if (classes[i] == 'background'):
        continue
    print('Current class: '+str(classes[i]))
    allIndices = findIndicesForIoU(classes[i])
    o50Sum = 0
    contained = 0
    ious, midPointContained = calculate_IoU(allIndices)
    for iou in ious:
        if iou >= 0.50:
            o50Sum += 1
    for mid in midPointContained:
        if mid:
            contained += 1
    o50Sum /= len(ious)
    contained /= len(ious)
    
    print('Average IoU for '+str(classes[i])+': '+str(np.mean(ious))+',      Percentage above 0.50 in IoU: '+str(o50Sum)+',     Percentage with midpoint in bb: '+str(contained))

Current class: carrots
Average IoU for carrots: 0.7047497713627167,      Percentage above 0.50 in IoU: 0.9315068493150684,     Percentage with midpoint in bb: 1.0
Current class: ketchup
Average IoU for ketchup: 0.638409143521245,      Percentage above 0.50 in IoU: 0.796969696969697,     Percentage with midpoint in bb: 1.0
Current class: arm
Average IoU for arm: 0.4853210006159183,      Percentage above 0.50 in IoU: 0.47474747474747475,     Percentage with midpoint in bb: 0.9932659932659933
Current class: bun
Average IoU for bun: 0.8260153971389403,      Percentage above 0.50 in IoU: 0.9940119760479041,     Percentage with midpoint in bb: 1.0
Current class: catfood_beef
Average IoU for catfood_beef: 0.4025470176679984,      Percentage above 0.50 in IoU: 0.3856655290102389,     Percentage with midpoint in bb: 0.9249146757679181
Current class: potato
Average IoU for potato: 0.7134413602982578,      Percentage above 0.50 in IoU: 0.98989898989899,     Percentage with midpoint in bb: 1.0
Cur

In [14]:
map_str = '/home/emil/Documents/github/BSc-PRO/dataset3/res_still/test'
classes = os.listdir(map_str)
allIndices = []
for i in range(len(classes)):
    if (classes[i] == 'background'):
        continue
    print('Current class: '+str(classes[i]))
    subIndices = findIndicesForIoU(classes[i])
    for indx in subIndices:
        allIndices.append(indx)

ious, midPointContained = calculate_IoU(allIndices)
o50Sum = 0
for iou in ious:
        if iou >= 0.50:
            o50Sum += 1
contained = 0
for mid in midPointContained:
    if mid:
        contained += 1
o50Sum /= len(ious)
contained /= len(ious)
print('Average IoU for Backprojection on all classes: '+str(np.mean(ious))+',       Percentage above 0.50 in IoU: '+str(o50Sum)+',     Percentage with midpoint in bb: '+str(contained))

Current class: carrots
Current class: ketchup
Current class: arm
Current class: bun
Current class: catfood_beef
Current class: potato
Current class: catfood_salmon
Average IoU for Backprojection on all classes: 0.5786200872887928,       Percentage above 0.50 in IoU: 0.690121155638397,     Percentage with midpoint in bb: 0.9645852749301025


|Class             | Average IoU | % Above 0.50 IoU | % With midpoint inside bb |
|:----------------:|:-----------:|:----------------:|:-------------------------:|
| Carrots          | 0.7047      |      0.9315      |        1.0000             |
| Ketchup          | 0.6384      |      0.7969      |        1.0000             |
| Arm              | 0.4853      |      0.4747      |        0.9932             |
| Bun              | 0.8260      |      0.9940      |        1.0000             |
| Cat food beef    | 0.4025      |      0.3856      |        0.9249             |
| Potato           | 0.7134      |      0.9898      |        1.0000             |
| Cat food salmon  | 0.2488      |      0.2178      |        0.8283             |
| **All classes**  | 0.5786      |      0.6901      |        0.9645             |

# Template Matching

In [6]:
from template_matching.template_matching import removeBackground, getRegionOfInterest, findTemplate

In [7]:
path = '/home/emil/Documents/github/BSc-PRO/Localization/template_matching/bgd_mask.jpg'
bgd_mask = cv2.imread(path, cv2.IMREAD_COLOR)

path_templates = [
    '/home/emil/Documents/github/BSc-PRO/Localization/template_matching/templates/template_potato.jpg',
    '/home/emil/Documents/github/BSc-PRO/Localization/template_matching/templates/template_arm.jpg',
    '/home/emil/Documents/github/BSc-PRO/Localization/template_matching/templates/template_cat_beef.jpg',
    '/home/emil/Documents/github/BSc-PRO/Localization/template_matching/templates/template_cat_sal.jpg',
    '/home/emil/Documents/github/BSc-PRO/Localization/template_matching/templates/template_bun.jpg',
    '/home/emil/Documents/github/BSc-PRO/Localization/template_matching/templates/template_carrot.jpg',
    '/home/emil/Documents/github/BSc-PRO/Localization/template_matching/templates/template_ketchup.jpg'
]
templates = [cv2.imread(temp, cv2.IMREAD_COLOR) for temp in path_templates]
def calculate_IoU(indices):
    combined = combine_labels_images()
    #print('Amount of combined objects: '+str(len(combined)))
    iou = []
    midPointContained = []
    progress = 0
    for indx in indices:
        current = combined[indx]

        img = cv2.imread(current[1])

        x = []
        map_str = '/home/emil/Documents/github/BSc-PRO/dataset3/res_still/test'
        classes = os.listdir(map_str)
        for c in classes:
            label = current[2]['Label']
            if c in label:
                ground_truth = c
                (x1, y1) = ( label[c][0]['geometry'][0]['x'],label[c][0]['geometry'][0]['y'] )
                (x2, y2) = ( label[c][0]['geometry'][1]['x'],label[c][0]['geometry'][1]['y'] )
                (x3, y3) = ( label[c][0]['geometry'][2]['x'],label[c][0]['geometry'][2]['y'] )
                (x4, y4) = ( label[c][0]['geometry'][3]['x'],label[c][0]['geometry'][3]['y'] )
                x = [x1, x2, x3, x4]
                y = [y1, y2, y3, y4]

        (xUL, yUL) = (np.min(x),np.min(y))
        (xBR, yBR) = (np.max(x),np.max(y))

        box = [xUL, yUL, xBR, yBR]
        
        # RESET LISTS FOR TEMPLATE MATCHING
        rois = []
        values = []
        cnts = []
        
        img = removeBackground(img, bgd_mask)
        for i in range(len(templates)):
            template = templates[i]
            roi = getRegionOfInterest(category=i,
                                          templ=template,
                                          src=img)
            rois.append(roi)
            (x_left, x_right, y_up, y_down) = roi
            roi = img[y_up : y_down, x_left : x_right]

            region = img[y_up : y_down, x_left : x_right]

            # Find template in region
            value, cnt = findTemplate(category=i,
                                      templ=template,
                                      src=roi)
            cnts.append(cnt)
            (x, y, width, height) = cnt
            cnt = img[y : y + height, x : x + width]

            values.append(value)

        index = np.argmax(values)
        
        (x_left, x_right, y_up, y_down) = rois[index]
        (x, y, width, height) = cnts[index]
        
        (pXUL, pYUL) = (x + x_left, y + y_up)
        (pXBR, pYBR) = (x + width + x_left, y + height + y_up)
        
        
        boxPred = [pXUL, pYUL, pXBR, pYBR]

        iou.append(IoU(box, boxPred))
        progress += 1
        if progress % 50 == 0:
            print('Progress: '+str(progress)+'/'+str(len(indices)))
            
        (midX, midY) = (pXUL + int((pXBR - pXUL)/2) , pYUL + int((pYBR - pYUL)/2) )
        if midX >=xUL and midX <= xBR and midY >= yUL and midY <= yBR:
            midPointContained.append(True)
        else:
            midPointContained.append(False)
    
    #print("IoU: "+str(iou))
    return iou, midPointContained

# Results for Template Matching **without** removal of background

In [20]:
map_str = '/home/emil/Documents/github/BSc-PRO/dataset3/res_still/test'
classes = os.listdir(map_str)
total_iou = []
total_mid = []
for i in range(len(classes)):
    if (classes[i] == 'background'):
        continue
    print('Current class: '+str(classes[i]))
    allIndices = findIndicesForIoU(classes[i])
    o50Sum = 0
    contained = 0
    ious, midPointContained = calculate_IoU(allIndices)
    for iou in ious:
        total_iou.append(iou)
        if iou >= 0.50:
            o50Sum += 1
    o50Sum /= len(ious)
    
    for mid in midPointContained:
        total_mid.append(mid)
        if mid:
            contained += 1
    contained /= len(ious)
    
    print('Average IoU for '+str(classes[i])+': '+str(np.mean(ious))+',      Percentage above 0.50 in IoU: '+str(o50Sum)+',     Percentage with midpoint in bb: '+str(contained))

total_50sum = 0
total_contained = 0
for tiou in total_iou:
    if tiou >= 0.50:
        total_50sum += 1
total_50sum /= len(total_iou)
for mid in total_mid:
    if mid:
        total_contained += 1
total_contained /= len(total_iou)
    
print('Average IoU for all classes: '+str(np.mean(total_iou))+',      Percentage above 0.50 in IoU: '+str(total_50sum)+',     Percentage with midpoint in bb: '+str(total_contained))

Current class: carrots
Progress: 50/292
Progress: 100/292
Progress: 150/292
Progress: 200/292
Progress: 250/292
Average IoU for carrots: 0.43763185207903227,      Percentage above 0.50 in IoU: 0.3561643835616438,     Percentage with midpoint in bb: 0.910958904109589
Current class: ketchup
Progress: 50/330
Progress: 100/330
Progress: 150/330
Progress: 200/330
Progress: 250/330
Progress: 300/330
Average IoU for ketchup: 0.09208148902637903,      Percentage above 0.50 in IoU: 0.045454545454545456,     Percentage with midpoint in bb: 0.20303030303030303
Current class: arm
Progress: 50/297
Progress: 100/297
Progress: 150/297
Progress: 200/297
Progress: 250/297
Average IoU for arm: 0.2670146311177735,      Percentage above 0.50 in IoU: 0.1414141414141414,     Percentage with midpoint in bb: 0.67003367003367
Current class: bun
Progress: 50/334
Progress: 100/334
Progress: 150/334
Progress: 200/334
Progress: 250/334
Progress: 300/334
Average IoU for bun: 0.41442871929475883,      Percentage abo

NameError: name 'total_ious' is not defined

In [21]:
total_50sum = 0
total_contained = 0
for tiou in total_iou:
    if tiou >= 0.50:
        total_50sum += 1
total_50sum /= len(total_iou)
for mid in total_mid:
    if mid:
        total_contained += 1
total_contained /= len(total_iou)
    
print('Average IoU for all classes: '+str(np.mean(total_iou))+',      Percentage above 0.50 in IoU: '+str(total_50sum)+',     Percentage with midpoint in bb: '+str(total_contained))

Average IoU for all classes: 0.24255086143353344,      Percentage above 0.50 in IoU: 0.15144454799627213,     Percentage with midpoint in bb: 0.538210624417521


|Class             | Average IoU | % Above 0.50 IoU | % With midpoint inside bb |
|:----------------:|:-----------:|:----------------:|:-------------------------:|
| Carrots          | 0.4376      |      0.3561      |        0.9109             |
| Ketchup          | 0.0920      |      0.0454      |        0.2030             |
| Arm              | 0.2670      |      0.1414      |        0.6700             |
| Bun              | 0.4144      |      0.1706      |        0.9161             |
| Cat food beef    | 0.0345      |      0.0000      |        0.0614             |
| Potato           | 0.4290      |      0.3602      |        0.9124             |
| Cat food salmon  | 0.0233      |      0.0000      |        0.0594             |
| **All classes**  | 0.2425      |      0.1514      |        0.5382             |

# Results for Template Matching **with** removal of background

In [8]:
map_str = '/home/emil/Documents/github/BSc-PRO/dataset3/res_still/test'
classes = os.listdir(map_str)
total_iou = []
total_mid = []
for i in range(len(classes)):
    if (classes[i] == 'background'):
        continue
    print('Current class: '+str(classes[i]))
    allIndices = findIndicesForIoU(classes[i])
    o50Sum = 0
    contained = 0
    ious, midPointContained = calculate_IoU(allIndices)
    for iou in ious:
        total_iou.append(iou)
        if iou >= 0.50:
            o50Sum += 1
    o50Sum /= len(ious)
    
    for mid in midPointContained:
        total_mid.append(mid)
        if mid:
            contained += 1
    contained /= len(ious)
    
    print('Average IoU for '+str(classes[i])+': '+str(np.mean(ious))+',      Percentage above 0.50 in IoU: '+str(o50Sum)+',     Percentage with midpoint in bb: '+str(contained))

total_50sum = 0
total_contained = 0
for tiou in total_iou:
    if tiou >= 0.50:
        total_50sum += 1
total_50sum /= len(total_iou)
for mid in total_mid:
    if mid:
        total_contained += 1
total_contained /= len(total_iou)
    
print('Average IoU for all classes: '+str(np.mean(total_iou))+',      Percentage above 0.50 in IoU: '+str(total_50sum)+',     Percentage with midpoint in bb: '+str(total_contained))

Current class: carrots
Progress: 50/292
Progress: 100/292
Progress: 150/292
Progress: 200/292
Progress: 250/292
Average IoU for carrots: 0.5425660270343416,      Percentage above 0.50 in IoU: 0.6095890410958904,     Percentage with midpoint in bb: 1.0
Current class: ketchup
Progress: 50/330
Progress: 100/330
Progress: 150/330
Progress: 200/330
Progress: 250/330
Progress: 300/330
Average IoU for ketchup: 0.5156132819124456,      Percentage above 0.50 in IoU: 0.49696969696969695,     Percentage with midpoint in bb: 0.9757575757575757
Current class: arm
Progress: 50/297
Progress: 100/297
Progress: 150/297
Progress: 200/297
Progress: 250/297
Average IoU for arm: 0.3665115191015087,      Percentage above 0.50 in IoU: 0.15824915824915825,     Percentage with midpoint in bb: 0.9764309764309764
Current class: bun
Progress: 50/334
Progress: 100/334
Progress: 150/334
Progress: 200/334
Progress: 250/334
Progress: 300/334
Average IoU for bun: 0.5104894735075904,      Percentage above 0.50 in IoU: 

| Class            | Average IoU | % Above 0.50 IoU | % With midpoint inside bb |
|:----------------:|:-----------:|:----------------:|:-------------------------:|
| Carrots          | 0.5425      |      0.6095      |        1.0000             |
| Ketchup          | 0.5156      |      0.4969      |        0.9757             |
| Arm              | 0.3665      |      0.1582      |        0.9764             |
| Bun              | 0.5104      |      0.4850      |        1.0000             |
| Cat food beef    | 0.1907      |      0.0000      |        0.9146             |
| Potato           | 0.5240      |      0.5286      |        1.0000             |
| Cat food salmon  | 0.1745      |      0.0000      |        0.8712             |
| **All classes**  | 0.4065      |      0.3299      |        0.9631             |